# Setup

In [18]:
from typing import Any
import json
import sys
sys.path.append('..')
from connectors.msgraph import Msgraph
mg = Msgraph()

In [2]:
# Tranform list of tuples to dict
def tuple_to_dict(tuple_list:list[tuple[str,Any]])->dict[str, Any]:
    return dict((key, value) for key, value in tuple_list)

# Get users data

## Get all users

In [13]:
(await mg.query("https://graph.microsoft.com/v1.0/users?$select=id,assignedLicenses,userType,userPrincipalName,accountEnabled,department,usageLocation,country"))[0][20]

MsGraph: https://graph.microsoft.com/v1.0/users?$select=id,assignedLicenses,userType,userPrincipalName,accountEnabled,department,usageLocation,country runned sucesfully
MsGraph: https://graph.microsoft.com/v1.0/users?$select=id%2cassignedLicenses%2cuserType%2cuserPrincipalName%2caccountEnabled%2cdepartment%2cusageLocation%2ccountry&$skiptoken=RFNwdAIAAQAAAD06Y2NhYnJlcmFfYmFuZXNjby5jb20jRVhUI0B0YWJhY2FsZXJhY2lnYXJyb3cub25taWNyb3NvZnQuY29tKVVzZXJfODE2MDY2YzAtOWM5MS00MTNmLThhYTAtODZhOGU3YzJlM2JluQAAAAAAAAAAAAA runned sucesfully
MsGraph: https://graph.microsoft.com/v1.0/users?$select=id%2cassignedLicenses%2cuserType%2cuserPrincipalName%2caccountEnabled%2cdepartment%2cusageLocation%2ccountry&$skiptoken=RFNwdAIAAEI6Q2RpZXR6X2FpcnpvbmVjb250cm9sLmNvbSNFWFQjQHRhYmFjYWxlcmFjaWdhcnJvdy5vbm1pY3Jvc29mdC5jb20pVXNlcl85NTY4NjQ2Ni1jNmZkLTQ5MjYtOTA4ZC0zMzBmNjA0MGYxYzUANzpFeGNoYW5nZV9PbmxpbmUtQXBwbGljYXRpb25BY2NvdW50QHRhYmFjYWxlcmFjaWdhci5jb20pVXNlcl9kYjdlOWQ1OC1kZDQwLTQ2ZGYtODE0Mi1iN2NlYzQ1MzJlODe5AAAAA

{'id': '8f472cc3-994d-4fd6-943d-6fb4d77010aa',
 'userType': 'Member',
 'userPrincipalName': 'aleandro.garcia@tabacaleracigar.com',
 'accountEnabled': True,
 'department': 'Box Plant',
 'usageLocation': 'DO',
 'country': 'República Dominicana',
 'assignedLicenses': [{'disabledPlans': [],
   'skuId': '05e9a617-0261-4cee-bb44-138d3ef5d965'},
  {'disabledPlans': [], 'skuId': '26124093-3d78-432b-b5dc-48bf992543d5'}]}

# Get all apps from devices

**URL to get all the devices**

GET https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/

## Get the first device which has the Windows OS

In [7]:
# id = list(filter(lambda x: x["operatingSystem"]=="Windows", await mg.query("https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/")))[0]["id"]
# print(id)
id = "7fe91247-dea9-4acc-9940-70310f694af1"

## Get all the APPS from the first windows device

### Using the MsGraph v1.0 🚫

In [8]:
await mg.query('https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/{id}/detectedApps')

MsGraph: ERROR https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/{id}/detectedApps failed
	BadRequest -- Resource not found for the segment 'detectedApps'.


([], False)

### Using the Msgraph Beta ✅

In [9]:
# Best solution for now to get the apps of the devices
await mg.query(f'https://graph.microsoft.com/beta/deviceManagement/managedDevices/{id}/detectedApps')

MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/7fe91247-dea9-4acc-9940-70310f694af1/detectedApps runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/7fe91247-dea9-4acc-9940-70310f694af1/detectedApps?$skip=50 runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/7fe91247-dea9-4acc-9940-70310f694af1/detectedApps?$skip=100 runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/7fe91247-dea9-4acc-9940-70310f694af1/detectedApps?$skip=150 runned sucesfully


([{'id': '0000f6937aa25758aa9d97d4d206a8ba9ea00000ffff',
   'displayName': '1Password',
   'version': '8.10.26',
   'sizeInByte': 0,
   'deviceCount': 0,
   'publisher': '',
   'platform': 'windows'},
  {'id': '35907b2b9fa9f3dc1b484bd5494b9f8e9b6a570d5010636f1ec6b829873e2851',
   'displayName': '22490Automattic.Simplenote',
   'version': '2.21.0.0',
   'sizeInByte': 0,
   'deviceCount': 0,
   'publisher': 'CN=E2E5A157-746D-4B04-9116-ABE5CB928306',
   'platform': 'windows'},
  {'id': 'de7062c38cd1ec337657b3118d4fdc60f3b3aea8b6d42eb790c8e590d17dceb8',
   'displayName': '4505Fortemedia.FMAPOControl',
   'version': '1.0.38.0',
   'sizeInByte': 0,
   'deviceCount': 0,
   'publisher': 'CN=DC696A1A-C35B-41EC-9C89-CC577F52CD81',
   'platform': 'windows'},
  {'id': '3287e0723d6af204f3d22c220c7232f2207cde5c6a34f317ed44ba0f01208079',
   'displayName': '5319275A.WhatsAppDesktop',
   'version': '2.2401.5.0',
   'sizeInByte': 0,
   'deviceCount': 0,
   'publisher': 'CN=24803D75-212C-471A-BC57-9EF86A

In [ ]:
await mg.query(f'https://graph.microsoft.com/beta/deviceManagement/managedDevices/{id}')

MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/7f1802c5-a7b4-4863-9e46-aaa64683ffc6 runned sucesfully


[]

## 🚫 Get all the detected apps from all devices with just one URL 

In [14]:
# get all decices with the detected apps -- NOT WORKING --
list(filter(lambda x: x["operatingSystem"]=="Windows" and x["detectedApps"] != [], (await mg.query("https://graph.microsoft.com/beta/deviceManagement/managedDevices?$expand=detectedApps"))[0]))

MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices?$expand=detectedApps runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices?$expand=detectedApps&$skiptoken=LastDeviceName%3d%27TSL-LP-3MXMP13%27%2cLastDeviceId%3d%27a3b43041-8ae1-468d-9d02-36aabad72b92%27 runned sucesfully


[]

⚠️ All the devices return a empty list of the apps they have.

## ✅ Get all the windows apps from all the devices, consulting from each device 

In [17]:
devices = list((filter(lambda x: x["operatingSystem"]=="Windows", (await mg.query("https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/"))[0])))
fields = ["id", "deviceName", "userId", "userDisplayName", "emailAddress", "operatingSystem", "osVersion"]

devices = list(map(lambda x: tuple_to_dict(list(map(lambda y: (y, x[y]), fields))), devices))[:50]

# apps = list(map(lambda x: await mg.query(f'https://graph.microsoft.com/beta/deviceManagement/managedDevices/{x["id"]}/detectedApps'), devices))
apps = []
for index, device in enumerate(devices):
    id = device["id"]
    device_apps = (await mg.query(f'https://graph.microsoft.com/beta/deviceManagement/managedDevices/{id}/detectedApps'))[0]
    for device_app in device_apps:
        device_app["deviceDetails"] = device
        apps.append(device_app)
print(apps)

MsGraph: https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/ runned sucesfully
MsGraph: https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/?$skiptoken=LastDeviceName%3d%27TSL-LP-3MXMP13%27%2cLastDeviceId%3d%27a3b43041-8ae1-468d-9d02-36aabad72b92%27 runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/d0d5489f-bc7a-4548-af9b-89803d4385ba/detectedApps runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/e10ea415-dfb2-4ca2-9fd8-277741e3d40c/detectedApps runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/e10ea415-dfb2-4ca2-9fd8-277741e3d40c/detectedApps?$skip=50 runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/a5254324-1600-45fc-913a-df65d5926607/detectedApps runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/a5254324-1600-45fc-913a-df65d5926607/detectedApps?$ski

Device fields
```py
{'id': 'e10ea415-dfb2-4ca2-9fd8-277741e3d40c',
 'userId': '1d7984d4-ba2f-4f98-800a-00b7a5dff323',
 'deviceName': 'DESKTOP-8TVV75J',
 'managedDeviceOwnerType': 'company',
 'enrolledDateTime': '2024-02-13T19:00:55Z',
 'lastSyncDateTime': '2024-02-13T20:35:50Z',
 'operatingSystem': 'Windows',
 'complianceState': 'compliant',
 'jailBroken': 'Unknown',
 'managementAgent': 'mdm',
 'osVersion': '10.0.19045.3930',
 'easActivated': False,
 'easDeviceId': '3EAD192E17525817885A674933310CA4',
 'easActivationDateTime': '0001-01-01T00:00:00Z',
 'azureADRegistered': True,
 'deviceEnrollmentType': 'windowsAzureADJoin',
 'activationLockBypassCode': None,
 'emailAddress': 'Manuela.Guerrero@tabacaleracigar.com',
 'azureADDeviceId': 'f1feceb6-d974-4db9-a0e3-0070623e3bce',
 'deviceRegistrationState': 'registered',
 'deviceCategoryDisplayName': 'Unknown',
 'isSupervised': False,
 'exchangeLastSuccessfulSyncDateTime': '0001-01-01T00:00:00Z',
 'exchangeAccessState': 'none',
 'exchangeAccessStateReason': 'none',
 'remoteAssistanceSessionUrl': None,
 'remoteAssistanceSessionErrorDetails': None,
 'isEncrypted': True,
 'userPrincipalName': 'Manuela.Guerrero@tabacaleracigar.com',
 'model': 'OptiPlex 3080',
 'manufacturer': 'Dell Inc.',
 'imei': '',
 'complianceGracePeriodExpirationDateTime': '9999-12-31T23:59:59Z',
 'serialNumber': '7CGMMH3',
 'phoneNumber': '',
 'androidSecurityPatchLevel': '',
 'userDisplayName': 'Guerrero, Manuela',
 'configurationManagerClientEnabledFeatures': None,
 'wiFiMacAddress': '',
 'deviceHealthAttestationState': None,
 'subscriberCarrier': '',
 'meid': '',
 'totalStorageSpaceInBytes': 255349227520,
 'freeStorageSpaceInBytes': 208705421312,
 'managedDeviceName': 'Manuela.Guerrero_Windows_2/13/2024_7:00 PM',
 'partnerReportedThreatState': 'unknown',
 'requireUserEnrollmentApproval': None,
 'managementCertificateExpirationDate': '2025-02-10T15:48:21Z',
 'iccid': None,
 'udid': None,
 'notes': None,
 'ethernetMacAddress': None,
 'physicalMemoryInBytes': 0,
 'deviceActionResults': []}
```

In [ ]:
import json


print(json.dumps(apps[0:10], indent=4))

[
    {
        "id": "0000857cb2cc316ab122bf3b7473f89152bf00000a04",
        "displayName": "Adobe Acrobat Reader DC - Espa\u00f1ol",
        "version": "15.007.20033",
        "sizeInByte": 0,
        "deviceCount": 0,
        "publisher": "",
        "platform": "windows",
        "deviceDetails": {
            "id": "e10ea415-dfb2-4ca2-9fd8-277741e3d40c",
            "deviceName": "DESKTOP-8TVV75J",
            "userId": "1d7984d4-ba2f-4f98-800a-00b7a5dff323",
            "userDisplayName": "Guerrero, Manuela",
            "emailAddress": "Manuela.Guerrero@tabacaleracigar.com",
            "operatingSystem": "Windows",
            "osVersion": "10.0.19045.3930"
        }
    },
    {
        "id": "00002b7618fefbc31a83c577ac911f4eb3940000ffff",
        "displayName": "Aplicaciones de Microsoft 365 para empresas - es-es",
        "version": "16.0.17231.20236",
        "sizeInByte": 0,
        "deviceCount": 0,
        "publisher": "",
        "platform": "windows",
        "deviceDe

# Get Microsoft Licenses

In [ ]:
response = await mg.query("https://graph.microsoft.com/v1.0/subscribedSkus")

print(json.dumps(response[0], indent=2))

MsGraph: https://graph.microsoft.com/v1.0/subscribedSkus runned sucesfully
[
  {
    "accountName": "tabacaleracigarrow",
    "accountId": "1410a3bb-e96e-4aa9-88df-fbbea4077580",
    "appliesTo": "User",
    "capabilityStatus": "Enabled",
    "consumedUnits": 5,
    "id": "1410a3bb-e96e-4aa9-88df-fbbea4077580_c5928f49-12ba-48f7-ada3-0d743a3601d5",
    "skuId": "c5928f49-12ba-48f7-ada3-0d743a3601d5",
    "skuPartNumber": "VISIOCLIENT",
    "subscriptionIds": [
      "a4acabf6-b4af-4e6d-8fe9-bdd49e609b5d"
    ],
    "prepaidUnits": {
      "enabled": 5,
      "suspended": 0,
      "warning": 0,
      "lockedOut": 0
    },
    "servicePlans": [
      {
        "servicePlanId": "da792a53-cbc0-4184-a10d-e544dd34b3c1",
        "servicePlanName": "ONEDRIVE_BASIC",
        "provisioningStatus": "Success",
        "appliesTo": "User"
      },
      {
        "servicePlanId": "2bdbaf8f-738f-4ac7-9234-3c3ee2ce7d0f",
        "servicePlanName": "VISIOONLINE",
        "provisioningStatus": "Success"

# Get logins

## Using 1.0 and beta get the logins from a date

In [26]:
list(filter(lambda x: not x["isInteractive"], (await mg.query("https://graph.microsoft.com/v1.0/auditLogs/signIns?$filter=createdDateTime ge 2024-02-25T00:00:00Z"))[0]))

MsGraph: https://graph.microsoft.com/v1.0/auditLogs/signIns?$filter=createdDateTime ge 2024-02-25T00:00:00Z runned sucesfully
MsGraph: https://graph.microsoft.com/v1.0/auditLogs/signIns?$filter=createdDateTime+ge+2024-02-25T00%3a00%3a00Z&$skiptoken=fee35990f42840dfa8877631fb6463b970dc03f029bda7d1366d3c1d183959d8 runned sucesfully
MsGraph: https://graph.microsoft.com/v1.0/auditLogs/signIns?$filter=createdDateTime+ge+2024-02-25T00%3a00%3a00Z&$skiptoken=93c78ea778bcf5533e816d4d80a2d670d28ca810c82cbffd91dda622477b2832 runned sucesfully
MsGraph: https://graph.microsoft.com/v1.0/auditLogs/signIns?$filter=createdDateTime+ge+2024-02-25T00%3a00%3a00Z&$skiptoken=7bcaa565c99cb0bbe3698a1ac5e65731ca36d102976a409f9963e86f0d34179c runned sucesfully
MsGraph: https://graph.microsoft.com/v1.0/auditLogs/signIns?$filter=createdDateTime+ge+2024-02-25T00%3a00%3a00Z&$skiptoken=727ba9993988c385119d11b55618aba26f87c813852ecdb4a8d76e69c6ec22ae runned sucesfully
MsGraph: https://graph.microsoft.com/v1.0/auditLog

[]

In [32]:
list(filter(lambda x: not x["isInteractive"], (await mg.query("https://graph.microsoft.com/beta/auditLogs/signIns?$filter=createdDateTime ge 2024-02-25T00:00:00Z"))[0]))

MsGraph: https://graph.microsoft.com/beta/auditLogs/signIns?$filter=createdDateTime ge 2024-02-25T00:00:00Z runned sucesfully
MsGraph: https://graph.microsoft.com/beta/auditLogs/signIns?$filter=createdDateTime+ge+2024-02-25T00%3a00%3a00Z&$skiptoken=4cf692030c8392843310daddf0f30d11ee1963197b16734d859496c7a517ea92 runned sucesfully
MsGraph: https://graph.microsoft.com/beta/auditLogs/signIns?$filter=createdDateTime+ge+2024-02-25T00%3a00%3a00Z&$skiptoken=80dd26b51f5cf7f088b4dfde40d8baf0594dfb648e6ee9032f8c59740f8f562c runned sucesfully
MsGraph: https://graph.microsoft.com/beta/auditLogs/signIns?$filter=createdDateTime+ge+2024-02-25T00%3a00%3a00Z&$skiptoken=2fff11f059eaee8eb20ac29c881ec4db9887c51c6cc9b703af4ea5e2e69ab0b9 runned sucesfully
MsGraph: https://graph.microsoft.com/beta/auditLogs/signIns?$filter=createdDateTime+ge+2024-02-25T00%3a00%3a00Z&$skiptoken=a71fe05ebd371678ddd0ef19560489edf94d351ca879bcc5e34d8bd726996a0d runned sucesfully
MsGraph: https://graph.microsoft.com/beta/auditLog

[]

## filtering that is not interactive. Empty list

In [30]:
list(filter(lambda x: not x["isInteractive"], (await mg.query("https://graph.microsoft.com/v1.0/auditLogs/signIns?$filter=createdDateTime ge 2024-02-25T00:00:00Z and isInteractive eq false"))[0]))

MsGraph: https://graph.microsoft.com/v1.0/auditLogs/signIns?$filter=createdDateTime ge 2024-02-25T00:00:00Z and isInteractive eq false runned sucesfully


[]

In [31]:
list(filter(lambda x: not x["isInteractive"], (await mg.query("https://graph.microsoft.com/beta/auditLogs/signIns?$filter=createdDateTime ge 2024-02-25T00:00:00Z and isInteractive eq false"))[0]))

MsGraph: https://graph.microsoft.com/beta/auditLogs/signIns?$filter=createdDateTime ge 2024-02-25T00:00:00Z and isInteractive eq false runned sucesfully


[]

## Other link

In [43]:
(await mg.query("https://graph.microsoft.com/beta/auditLogs/signIns?$filter=signInEventTypes/any(t: t eq 'nonInteractiveUser')&$top=10"))[0]

MsGraph: https://graph.microsoft.com/beta/auditLogs/signIns?$filter=signInEventTypes/any(t: t eq 'nonInteractiveUser')&$top=10 runned sucesfully
MsGraph: https://graph.microsoft.com/beta/auditLogs/signIns?$filter=signInEventTypes%2fany(t%3a+t+eq+%27nonInteractiveUser%27)&$top=10&$skiptoken=98fd05b38b76429faa5a3c375731bf63596c3cccd8f4b2697effc616cbfd585d runned sucesfully
MsGraph: https://graph.microsoft.com/beta/auditLogs/signIns?$filter=signInEventTypes%2fany(t%3a+t+eq+%27nonInteractiveUser%27)&$top=10&$skiptoken=77160ce664f89452d93da5f242f7a57810ef8a27481194ce66aab2249f75382a runned sucesfully
MsGraph: https://graph.microsoft.com/beta/auditLogs/signIns?$filter=signInEventTypes%2fany(t%3a+t+eq+%27nonInteractiveUser%27)&$top=10&$skiptoken=ece689ad1ca5a243c573cab254e66fdb12e4d9e31400725780f3b02085837066 runned sucesfully
MsGraph: https://graph.microsoft.com/beta/auditLogs/signIns?$filter=signInEventTypes%2fany(t%3a+t+eq+%27nonInteractiveUser%27)&$top=10&$skiptoken=1c7e1533cb1ca36e53fa7e

[{'id': '040fafb3-bd5e-4ae0-b6c1-7b5d64b3e300',
  'createdDateTime': '2024-03-01T09:22:55Z',
  'userDisplayName': 'Garrett Pinero, Mónica',
  'userPrincipalName': 'monica.garrett@tabacaleracigar.com',
  'userId': '1574f02d-bd66-4a1b-897e-f676daff989e',
  'appId': '0000000c-0000-0000-c000-000000000000',
  'appDisplayName': 'Microsoft App Access Panel',
  'ipAddress': '195.53.89.98',
  'ipAddressFromResourceProvider': None,
  'clientAppUsed': 'Browser',
  'userAgent': '',
  'correlationId': 'f8a7b2f6-3991-4525-85e8-cc320ab03ab9',
  'conditionalAccessStatus': 'success',
  'originalRequestId': '',
  'isInteractive': False,
  'tokenIssuerName': '',
  'tokenIssuerType': 'AzureAD',
  'clientCredentialType': 'clientAssertion',
  'processingTimeInMilliseconds': 60,
  'riskDetail': 'none',
  'riskLevelAggregated': 'none',
  'riskLevelDuringSignIn': 'none',
  'riskState': 'none',
  'riskEventTypes_v2': [],
  'resourceDisplayName': 'Microsoft password reset service',
  'resourceId': '93625bc8-bfe2